In [ ]:
# The helpers use the application default credentials.
from cmle_utils import get_models, get_model_versions, train_model, deploy_model, set_default, predict_json, build_and_upload_package, create_package

## run locally

### train

In [ ]:
import argparse
import tensorflow as tf

In [ ]:
def model_fn(features, labels, mode):
    global_step = tf.train.get_or_create_global_step()
    hidden = tf.layers.dense(features, 10, activation=tf.nn.relu)
    outputs = tf.layers.dense(hidden, 1)

    predictions = outputs
    loss = None
    train_op = None

    if mode == tf.estimator.ModeKeys.TRAIN:
        loss = tf.nn.l2_loss(outputs - labels)

        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05)
        train_op = optimizer.minimize(loss, global_step=global_step)

    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions,
        loss=loss,
        train_op=train_op
    )


In [ ]:
def train_input_fn():
    fake_features = tf.constant([[1], [2]], dtype=tf.float32)
    fake_labels = tf.constant([[3], [4]], dtype=tf.float32)
    ds = tf.data.Dataset.from_tensor_slices(
        (fake_features, fake_labels)
    )
    return ds.repeat().batch(10)

In [ ]:
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--job-dir',
        default='/tmp/cmle'
    )
    parser.add_argument(
        '--steps',
        default=300)

    args, unused_args = parser.parse_known_args()
    
    return args

In [ ]:
def train(args):
    estimator = tf.estimator.Estimator(model_fn, model_dir=args.job_dir)
    estimator.train(train_input_fn, steps=args.steps)
    
    return estimator

In [ ]:
args = parse_args()
print(args)

In [ ]:
estimator = train(args)

### predict

In [ ]:
predictions = estimator.predict(input_fn=train_input_fn)

In [ ]:
print(next(predictions))

## run on CMLE

### train

In [ ]:
execfile('cmle_utils.py')

In [ ]:
create_package(model_fn, train_input_fn, parse_args, train)

In [ ]:
# package and upload to gcs before we can train
# this requires having a setup.py in the directory
# this needs to be rerun everytime the trainer model is updated and right before calling train_model

package_path = build_and_upload_package('gs://.../package')

# this is needed in the job_spec
print(package_path)

In [ ]:
# train in the cloud
import time
import os

BUCKET = 'gs://.../test'
JOB_DIR = os.path.join(BUCKET, 'output')

training_inputs = {
    'scaleTier': 'BASIC',
    'packageUris': [package_path],
    'pythonModule': 'trainer.task',
    'region': 'us-central1',
    'jobDir': JOB_DIR,
    'runtimeVersion': '1.8',
    'args': [
        '--steps 1000'
    ]
}
job_spec = {'jobId': 'test_' + str(int(time.time())), 'trainingInput': training_inputs}

In [ ]:
PROJECT_ID = ...
train_model(PROJECT_ID, job_spec)

### deploy


### predict